In [1]:
import sys

sys.path.append('../../')

In [2]:
from ten.nanoparticle import Nanoparticle
from ten.aceptor import Aceptor
import ten.experiments as experiments

from ten.mechanisms import transferF90

In [3]:
print(transferF90.transfer_rate.__doc__)

rate = transfer_rate(exiton,aceptors,tau_d,r_forster,[m])

Wrapper for ``transfer_rate``.

Parameters
----------
exiton : input rank-1 array('d') with bounds (3)
aceptors : input rank-2 array('d') with bounds (m,3)
tau_d : input float
r_forster : input float

Other Parameters
----------------
m : input int, optional
    Default: shape(aceptors,0)

Returns
-------
rate : float



In [4]:
# Implementacion original en python

def transfer_py(exiton_pos, aceptors_pos, tau_d, r_forster):
    # constente usa para el calculo de k_et
    cte = r_forster**6/tau_d

    diff = exiton_pos - aceptors_pos
    component_square = diff*diff
    one_over_distance = 1/(component_square[:, 0] +
                           component_square[:, 1] +
                           component_square[:, 2])
    distance_6 = one_over_distance*one_over_distance*one_over_distance
    k = cte*sum(distance_6)
    return(k)

In [5]:
# Dopantes propios (intrisicos) de la NP
dopantes_propios = Aceptor(number=10, r_mechanisms=1, way='vol')
# Dopantes que le vamos a agregar a la NP
dopantes = Aceptor(number=500, r_mechanisms=3.14, way='vol')

# Nanoparticula
NP = Nanoparticle(r_param=[15, 0], tau_d=0.333, mean_path=80, epsilon=1, intrinsic_aceptors=dopantes_propios)
# Dopo la NP
NP.doped(dopantes)
# Genero el exiton
NP.excite('laser')

In [6]:
NP.exiton.position

array([-6.70457227, -1.27616068, -1.16347277])

In [7]:
NP.aceptors.position.shape

(500, 3)

In [8]:
transfer_py(NP.exiton.position, NP.aceptors.position, NP.tau_d, NP.aceptors.r_mechanisms)

36662.964238141038

In [9]:
transferF90.transfer_rate(NP.exiton.position, NP.aceptors.position, 
                          NP.tau_d, NP.aceptors.r_mechanisms)

36662.96423814104

In [10]:
%%timeit

for i in range(10000):
    out1 = transfer_py(NP.exiton.position, NP.aceptors.position, NP.tau_d, NP.aceptors.r_mechanisms)

1 loop, best of 3: 1.7 s per loop


In [11]:
%%timeit

for i in range(10000):
    out2 = transferF90.transfer_rate(NP.exiton.position, NP.aceptors.position, 
                                     NP.tau_d, NP.aceptors.r_mechanisms)

10 loops, best of 3: 135 ms per loop
